## Demo for SDK (v0.0.2)

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:
- [initialization](#initialization)
- [show available hardware info list](#show-available-hardware-information)
- [choose hardware config](#choose-hardware-config)
- [get `job_source_uri`](#get-job_source_uri)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()
from swan import SwanAPI

# dev_url = "https://swanhub-cali.swanchain.io"
# dev_url = "http://127.0.0.1:5008"

# Initialize the Swan Service
# dev
# swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment=dev_url)
# prod
swan_api = SwanAPI(api_key=os.getenv("API_KEY"))

from swan.contract.swan_contract import SwanContract

contract = SwanContract(os.getenv('PK'), swan_api.contract_info)

In [2]:
r = swan_api.contract_info
print(json.dumps(r, indent=2))

{
  "client_contract_address": "0x20a67c6Bea000fAf0BE862BB254F092abF0E5b98",
  "payment_contract_address": "0x5094A609Af5184d076Be2DF741820732126b4Fd2",
  "rpc_url": "https://rpc-atom-internal.swanchain.io",
  "swan_token_contract_address": "0x91B25A65b295F0405552A4bbB77879ab5e38166c"
}


### Show available hardware information

In [3]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['North Carolina-US',
   'Bashkortostan Republic-RU',
   'Kyiv City-UA',
   'Kowloon City-HK',
   'Tokyo-JP',
   'California-US',
   'Central and Western District-HK',
   'Quebec-CA',
   'North West-SG',
   'Kwai Tsing-HK',
   'Bavaria-DE',
   'Saxony-DE',
   'Guangdong-CN',
   'Kowloon-HK',
   'North Rhine-Westphalia-DE'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['North Carolina-US',
   'Bashkortostan Republic-RU',
   'Kyiv City-UA',
   'Kowloon City-HK',
   'Tokyo-JP',
   'California-US',
   'Central and Western District-HK',
   'Quebec-CA',
   'North West-SG',
   'Kwai Tsing-HK',
   'Bavaria-DE',
   'Guangdong-CN',
   'Kowloon-HK',
   'North Rhine-Westphalia-DE'],
  'price': '1.0',
  'status': 'available'},
 {'id': 4,
  'name': 'M1ae.large',
  'description': '

### Choose hardware config

choose the hardware with its config name or hardware id

in hardware config, `cfg_name` and `hardware_id` will be used in the steps to deploy task (create task and submit payment)

In [4]:
cfg_name = 'C1ae.medium' #"G1ae.medium"
hardware = [hardware for hardware in hardwares if hardware.name == cfg_name][0]
print(f"{hardware.name=}, {hardware.id=}, {hardware.region}")

hardware_id = hardware.id
print(f"The chosen {hardware_id=}")

hardware.name='C1ae.medium', hardware.id=1, ['North Carolina-US', 'Bashkortostan Republic-RU', 'Kyiv City-UA', 'Kowloon City-HK', 'Tokyo-JP', 'California-US', 'Central and Western District-HK', 'Quebec-CA', 'North West-SG', 'Kwai Tsing-HK', 'Bavaria-DE', 'Guangdong-CN', 'Kowloon-HK', 'North Rhine-Westphalia-DE']
The chosen hardware_id=1


In [5]:
hardware_id

1

### Get `job_source_uri`

We can use the `get_source_uri` interface to get a `job_source_uri` for creating task.

In [6]:

job_source_uri = swan_api.get_source_uri(
    repo_uri='https://github.com/alphaflows/tetris-docker-image.git',
    hardware_id=hardware_id,
    wallet_address=os.getenv('WALLET')
)

In [7]:
if job_source_uri and job_source_uri.get('data'):
    job_source_uri = job_source_uri['data']['job_source_uri']

In [8]:
job_source_uri

'https://data.mcs.lagrangedao.org/ipfs/QmNbszwFUG7ZCsi3NjdEDME8WzX6LL3GHVmMf8KDmyUrYG'

### Estimate Payment amount

In [9]:
duration_hour = 1 # hour
amount = contract.estimate_payment(hardware_id, duration_hour)
print(amount)

1000000000000000000


### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [10]:
duration=3600*duration_hour

result = swan_api.create_task(
    cfg_name=cfg_name, 
    region='North Carolina-US', 
    start_in=300, 
    duration=duration, 
    job_source_uri=job_source_uri,
    paid=contract._wei_to_swan(amount),
    wallet_address=os.getenv('WALLET'),
)
print(json.dumps(result, indent=2))
task_uuid = result['data']['task']['uuid']

{
  "data": {
    "task": {
      "created_at": "1714766359",
      "end_at": "1714769950",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "initialized",
      "task_detail_cid": "https://data.mcs.lagrangedao.org/ipfs/QmVkif9jcuPXnoFjcVx1V5AKAVBVdA1VDMUMRkKbBEPR5D",
      "tx_hash": null,
      "updated_at": "1714766359",
      "uuid": "d2ee1a94-f96f-4fb3-8a64-65d26314e000"
    }
  },
  "message": "Task_uuid initialized.",
  "status": "success"
}


### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [11]:
tx_hash = contract.submit_payment(task_uuid, hardware_id, duration)
print(tx_hash)

0x68b940951a5e9d7ccb8674b4342fab0de942a3aa64d5395d74f874f48cb7a58d


### Validate Payment to deploy task

This step will validate the payment and then make task eligible for assigning if validation successful

In [12]:
if result_validation := swan_api.validate_payment(
    tx_hash=tx_hash,
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

{'tx_hash': '0x68b940951a5e9d7ccb8674b4342fab0de942a3aa64d5395d74f874f48cb7a58d', 'task_uuid': 'd2ee1a94-f96f-4fb3-8a64-65d26314e000'}
{
  "data": {
    "task": {
      "created_at": "1714766359",
      "end_at": "1714769950",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "created",
      "task_detail_cid": "https://data.mcs.lagrangedao.org/ipfs/QmVkif9jcuPXnoFjcVx1V5AKAVBVdA1VDMUMRkKbBEPR5D",
      "tx_hash": null,
      "updated_at": "1714766390",
      "uuid": "d2ee1a94-f96f-4fb3-8a64-65d26314e000"
    }
  },
  "message": "Task payment validated successfully.",
  "status": "success"
}


### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [14]:
# Check task info
info = swan_api.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [],
    "jobs": [],
    "task": {
      "created_at": "1714766359",
      "end_at": "1714769950",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "accepting_bids",
      "task_detail_cid": "https://data.mcs.lagrangedao.org/ipfs/QmVkif9jcuPXnoFjcVx1V5AKAVBVdA1VDMUMRkKbBEPR5D",
      "tx_hash": null,
      "updated_at": "1714766411",
      "uuid": "d2ee1a94-f96f-4fb3-8a64-65d26314e000"
    }
  },
  "message": "fetch task info for task_uuid='d2ee1a94-f96f-4fb3-8a64-65d26314e000' successfully",
  "status": "success"
}


### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [ ]:
r = swan_api.get_real_url(task_uuid)
print(r)

['https://077dupt8wa.cp.filezoo.com.cn']


In [ ]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


IndexError: list index out of range